In [1]:
from src.models.linreg import LinReg

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

In [2]:
np.random.seed(69)
x = np.linspace(0, 5, 50)
z = np.linspace(3, 8, 50)
y = 1 + 3 * x + 5*z + np.random.normal(0, 2, 50)
data = pd.DataFrame({'outcome': y,
                     'x': x,
                     'z': z})

In [7]:
linreg = LinReg(df=data, outcome="outcome", independent=[ "x*z"])
linreg.independent_vars

['x*z', 'z', 'x']

In [8]:
linreg.data

,outcome,x,z,x*z
0,17.831014,0.000000,3.000000,0.000000
1,15.609243,0.102041,3.102041,0.316535
2,19.957243,0.204082,3.204082,0.653894
3,17.246179,0.306122,3.306122,1.012078
4,16.070335,0.408163,3.408163,1.391087
5,20.877085,0.510204,3.510204,1.790920
6,23.309690,0.612245,3.612245,2.211579
7,23.826884,0.714286,3.714286,2.653061
8,24.235700,0.816327,3.816327,3.115369
9,24.725722,0.918367,3.918367,3.598501


In [9]:
linreg.summary()

In [ ]:
linreg.summary(content_type='simple')

In [ ]:
string = '~1'

'~' in string